## Grab production results

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from openpyxl import Workbook
import xlsxwriter
import pandas.io.formats.excel
import glob
from pandas import ExcelWriter

In [2]:
# Requisite input

economies = ['01_AUS', '02_BD', '03_CDA', '04_CHL', '05_PRC', '06_HKC',
             '07_INA', '08_JPN', '09_ROK', '10_MAS', '11_MEX', '12_NZ',
             '13_PNG', '14_PE', '15_RP', '16_RUS', '17_SIN', '18_CT', '19_THA',
             '20_USA', '21_VN']

historical_years = list(map(str, range(2008, 2018, 1)))
future_years = list(map(str, range(2017, 2051, 1)))

In [3]:
# Read historcial data from projection results to work with for energy intensities

production_data = pd.DataFrame()

for economy in economies:
    interim_df = pd.read_excel('../data/input_data/0_production_results/production_projection_workbook.xlsx', skiprows = 4, nrows = 14, sheet_name = economy)
    transport = interim_df[interim_df['item_code_new'] == '13_5_transportation_equipment']
    not_transport = interim_df[interim_df['item_code_new'] != '13_5_transportation_equipment']
    agg_transport = transport.groupby(['economy', 'unit', 'item_code_new']).sum().assign(item = '8th_transport_production_projection').reset_index()
    interim_df = not_transport.append(agg_transport).reset_index(drop = True)
    interim_df['item_code_new'] = pd.Categorical(interim_df['item_code_new'], ['13_1_iron_and_steel',
                                             '13_2_chemical_incl__petrochemical',
                                             '13_3_nonferrous_metals',
                                             '13_4_nonmetallic_mineral_products', 
                                             '13_5_transportation_equipment', 
                                             '13_6_machinery', 
                                             '13_7_mining_and_quarrying',
                                             '13_8_food_beverages_and_tobacco', 
                                             '13_9_pulp_paper_and_printing', 
                                             '13_10_wood_and_wood_products',
                                             '13_11_construction',
                                             '13_12_textiles_and_leather',
                                             '13_13_nonspecified_industry'])
    interim_df = interim_df.sort_values('item_code_new')                                         
    production_data = production_data.append(interim_df)

production_data_historical = production_data.dropna(subset = ['economy']).reset_index(drop = True).reindex(columns = ['economy', 'item', 'unit', 'item_code_new'] + historical_years)

production_data_historical.to_csv('../results/post_projection/production_historical.csv', index = False)

production_data_estimates = production_data.dropna(subset = ['economy']).reset_index(drop = True).reindex(columns = ['economy', 'item', 'unit', 'item_code_new'] + future_years)

production_data_estimates.to_csv('../results/post_projection/production_estimates.csv', index = False)